In [1]:
pip install pandas openpyxl geopandas matplotlib

In [ ]:
import pandas as pd
import geopandas as gpd
from dataclasses import dataclass, field
from typing import List, Dict, Set

# Classes principales

@dataclass
class Infrastructure:
    id: str
    type: str  # 'infra_intacte' ou 'a_remplacer'
    longueur: float
    batiments_desservis: Set[str] = field(default_factory=set)
    nb_maisons_total: int = 0
    difficulte: float = 0.0
    est_reparee: bool = False
    phase_reparation: int = -1

    def calculer_difficulte(self):
        """Calcule la difficulté d'une infrastructure"""
        self.difficulte = self.longueur / self.nb_maisons_total if self.nb_maisons_total > 0 else float('inf')
        return self.difficulte

    def est_operationnelle(self):
        """Vérifie si l'infrastructure est utilisable"""
        return self.type == "infra_intacte" or self.est_reparee


@dataclass
class Batiment:
    id: str
    nb_maisons: int
    infrastructures: List[Infrastructure] = field(default_factory=list)
    difficulte_totale: float = 0.0
    phase: int = -1

    def calculer_difficulte(self):
        """Somme des difficultés des infras non réparées"""
        infras_non_op = [i for i in self.infrastructures if not i.est_operationnelle()]
        self.difficulte_totale = sum(i.difficulte for i in infras_non_op)
        return self.difficulte_totale

    def est_raccorde(self):
        return all(i.est_operationnelle() for i in self.infrastructures)



# Planificateur de raccordement

class PlanificateurRaccordement:
    def __init__(self, xlsx_path, shp_infra_path, shp_bat_path):
        self.xlsx_path = xlsx_path
        self.shp_infra_path = shp_infra_path
        self.shp_bat_path = shp_bat_path

        self.df_reseau = None
        self.gdf_infrastructures = None
        self.gdf_batiments = None

        self.infrastructures: Dict[str, Infrastructure] = {}
        self.batiments: Dict[str, Batiment] = {}
        self.plan_raccordement: List[Dict] = []

    # CHARGEMENT 
    def charger_donnees(self):
        print("Chargement des données...")
        self.df_reseau = pd.read_excel(self.xlsx_path)
        self.gdf_infrastructures = gpd.read_file(self.shp_infra_path)
        self.gdf_batiments = gpd.read_file(self.shp_bat_path)

        print(f"{len(self.df_reseau)} lignes de réseau chargées")

        # Créer les infrastructures
        for infra_id in self.df_reseau['infra_id'].unique():
            ligne = self.df_reseau[self.df_reseau['infra_id'] == infra_id].iloc[0]
            self.infrastructures[infra_id] = Infrastructure(
                id=infra_id,
                type=ligne['infra_type'],
                longueur=ligne['longueur']
            )

        # Créer les bâtiments
        for bat_id in self.df_reseau['id_batiment'].unique():
            data = self.df_reseau[self.df_reseau['id_batiment'] == bat_id]
            nb = data['nb_maisons'].iloc[0]
            bat = Batiment(id=bat_id, nb_maisons=nb)

            for _, row in data.iterrows():
                infra = self.infrastructures[row['infra_id']]
                bat.infrastructures.append(infra)
                infra.batiments_desservis.add(bat_id)
                infra.nb_maisons_total += nb

            self.batiments[bat_id] = bat

        # Calcul des difficultés
        for infra in self.infrastructures.values():
            infra.calculer_difficulte()
        for bat in self.batiments.values():
            bat.calculer_difficulte()

        print(f"{len(self.batiments)} bâtiments chargés")
        print(f"{len(self.infrastructures)} infrastructures chargées\n")

    # PLANIFICATION 
    def planifier_raccordement(self):
        print("Début de la planification...\n")

        phase = 1
        cout_total = 0

        # Identifier les bâtiments déjà raccordés
        for bat in self.batiments.values():
            if bat.est_raccorde():
                bat.phase = 0

        a_raccorder = [b for b in self.batiments.values() if b.phase == -1]
        print(f"{len(a_raccorder)} bâtiments à raccorder.\n")

        # Boucle principale
        while a_raccorder:
            for bat in a_raccorder:
                bat.calculer_difficulte()
            a_raccorder.sort(key=lambda x: x.difficulte_totale)

            batiment = a_raccorder.pop(0)
            batiment.phase = phase

            # Infras à réparer
            infras_a_reparer = [i for i in batiment.infrastructures if not i.est_operationnelle()]
            infras_uniques = list({i.id: i for i in infras_a_reparer}.values())

            cout_phase = sum(i.longueur for i in infras_uniques)
            cout_total += cout_phase

            for i in infras_uniques:
                i.est_reparee = True
                i.phase_reparation = phase

            self.plan_raccordement.append({
                "Phase": phase,
                "Bâtiment": batiment.id,
                "Maisons": batiment.nb_maisons,
                "Difficulté": round(batiment.difficulte_totale, 2),
                "Infras réparées": ", ".join(i.id for i in infras_uniques),
                "Coût phase (m)": round(cout_phase, 2),
                "Coût cumulé (m)": round(cout_total, 2)
            })
            phase += 1

        print("Planification terminée.")
        print(f"Nombre total de phases : {phase-1}")
        print(f"Coût total estimé : {round(cout_total,2)} m\n")

    # AFFICHAGE 
    def afficher_plan(self):
        """Affiche le plan sous forme de tableau"""
        df = pd.DataFrame(self.plan_raccordement)
        print("Tableau du plan de raccordement :\n")
        print(df.to_string(index=False))
        return df


# Utilisation du code

if __name__ == "__main__":
    xlsx_path = "reseau_en_arbre.xlsx"
    shp_infra_path = "infrastructures.shp"
    shp_bat_path = "batiments.shp"

    planif = PlanificateurRaccordement(xlsx_path, shp_infra_path, shp_bat_path)
    planif.charger_donnees()
    planif.planifier_raccordement()
    df_plan = planif.afficher_plan()

    # Sauvegarde du tableau dans un fichier Excel
    df_plan.to_excel("plan_raccordement_tableau.xlsx", index=False)
    print("\nTableau enregistré sous 'plan_raccordement_tableau.xlsx'")


In [ ]:
# Load the shapefiles to inspect their column names
try:
    gdf_infrastructures_cols = gpd.read_file("infrastructures.shp")
    print("Columns in infrastructures.shp:")
    print(gdf_infrastructures_cols.columns)

    gdf_batiments_cols = gpd.read_file("batiments.shp")
    print("\nColumns in batiments.shp:")
    print(gdf_batiments_cols.columns)
except Exception as e:
    print(f"Error loading shapefiles: {e}")